In [ ]:
import numpy as np

np.random.seed(123)
X = np.load('../generated_games/features-40k.npy')
Y = np.load('../generated_games/labels-40k.npy')
print("features", X.shape)
print("labels", Y.shape)

samples = X.shape[0]
size = 9
input_shape = (size, size, 1)

X = X.reshape(samples, size, size, 1)

train_samples = 10000
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

features (11705, 9, 9)
labels (11705, 81)
X_train (10000, 9, 9, 1)
Y_train (10000, 81)
X_test (1705, 9, 9, 1)
Y_test (1705, 81)


In [41]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input

model = Sequential()
model.add(Input(shape=(input_shape)))
model.add(Conv2D(filters=48,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Conv2D(48, (3, 3),
                 padding='same', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Flatten()) 
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(size * size, activation='softmax')) 
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 9, 9, 48)       │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 9, 9, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 9, 9, 48)       │        20,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 81)             │        41,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,545 (1.74 MB)

 Trainable params: 456,545 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
import time

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Record the start time
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=15,
          verbose=1,
          validation_data=(X_test, Y_test))
# Record the end time
end_time = time.time()
# Calculate and display the wall time
print(f"Wall time: {end_time - start_time:.6f} seconds")

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.0116 - loss: 4.4412 - val_accuracy: 0.0152 - val_loss: 4.3939
Epoch 2/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0130 - loss: 4.4118 - val_accuracy: 0.0135 - val_loss: 4.3909
Epoch 3/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0135 - loss: 4.4032 - val_accuracy: 0.0152 - val_loss: 4.3894
Epoch 4/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0131 - loss: 4.3955 - val_accuracy: 0.0147 - val_loss: 4.3879
Epoch 5/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.0175 - loss: 4.3915 - val_accuracy: 0.0129 - val_loss: 4.3864
Epoch 6/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0145 - loss: 4.3874 - val_accuracy: 0.0100 - val_loss: 4.3850
Epoch 7/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0172 - loss: 4.3848 - val_accuracy: 0.0129 - val_loss: 4.3839
Epoch 8/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0173 - loss: 4.3876 - val_accu

Starlight MacBook Air M2:

MacBookPro M1 Pro:
> Wall time: 38.298259 seconds

> Test loss: 4.374318599700928
> Test accuracy: 0.014076245948672295

In [43]:
test_board = np.array([[
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 0,  1, -1,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
]]).reshape(1, 9, 9)
print(test_board)
move_probs = model.predict(test_board)[0]
i = 0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

[[[ 0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0]
  [ 0  1 -1  1 -1  0  0  0  0]
  [ 0  1 -1  1 -1  0  0  0  0]
  [ 0  0  1 -1  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
0.009 0.011 0.011 0.011 0.011 0.012 0.011 0.012 0.009
0.011 0.012 0.013 0.013 0.014 0.013 0.013 0.013 0.011
0.011 0.013 0.013 0.013 0.013 0.014 0.013 0.013 0.011
0.011 0.013 0.013 0.014 0.014 0.014 0.014 0.013 0.012
0.011 0.013 0.014 0.014 0.014 0.014 0.014 0.013 0.012
0.011 0.013 0.014 0.014 0.014 0.014 0.013 0.013 0.011
0.011 0.013 0.014 0.014 0.013 0.014 0.013 0.013 0.011
0.011 0.013 0.013 0.013 0.013 0.013 0.013 0.012 0.011
0.009 0.011 0.011 0.012 0.012 0.012 0.010 0.011 0.009
